# Python SDK Tutorial
This notebook explains how to use the Python SDK of ModelBox and explains the major concepts and how to use the API independent of any Deep Learning Framework. Please follow the PyTorch notebook to see how the SDK can be integrated with a PyTorch trainer. 



## Install ModelBox SDK

In [25]:
pip install modelbox==0.0.4 grpcio

Note: you may need to restart the kernel to use updated packages.


## Initialize the ModelBox Client
First, we initialize the client by pointing it to the address of the ModelBox Server

In [1]:
from modelbox.modelbox import ModelBox, MLFramework, Artifact, ArtifactMime, MetricValue

mbox = ModelBox(addr="172.21.0.2:8085")

## Create an Experiment 
Once we have a client, we can start using it to create a new Experiment to train a model or track an existing pre-trained model. Let us first see how to create an experiment. We are going to create an experiment to train a Wav2Vec Model with Pytorch and store it in a namespace called *langtech*. If you are using an experiment management service like Weights and Biases or Nepute, you could associate the ID from that service with modelbox to create a lineage.

In [2]:
experiment = mbox.new_experiment("wav2vec", "owner@pytorch.org", "langtech", "extern123", MLFramework.PYTORCH)
experiment.id

'e71a25a8d3d481463c281f083f5e2671ea2896bc'

The above code is going to create a new experiment and give us an ID. You can list the experiments of a namespace by -

In [5]:
mbox.list_experiments(namespace="langtech")

ListExperimentsResponse(experiments=[Experiment(id='e71a25a8d3d481463c281f083f5e2671ea2896bc', name='wav2vec', owner='owner@pytorch.org', namespace='langtech', external_id='', created_at=seconds: 1663835847
, updated_at=seconds: 1663835847
, framework=<MLFramework.PYTORCH: 2>)])

#### Adding metadata
Metadata can be added to any of the objects in ModelBox after they have been created. For example, once an experiment is created, metadata can be added and listed at any stage -

In [8]:
experiment.update_metadata("foo/bar", 12)
experiment.metadata()

ListMetadataResponse(metadata={'foo/bar': number_value: 12
})

## Working with Checkpoints
Once we have an experiment we can create model checkpoints from the trainers. Let's assume the file stored in assets/mnist_checkpoint1.pt is a checkpoint created by the trainer. We will now associate this checkpoint with ModelBox.

We could either track the path of the checkpoint or upload the blob and let ModelBox store it in the configured blob store. The benefit of letting ModelBox store the checkpoint is that the trainer doesn't need to have access to the blob store directly. However, in some cases, it's more optimal to have the trainer store the blob directly when the path to IO to the blob store from the trainer is much faster.

In [10]:
import os
checkpoint_path = os.path.abspath('mnist_cnn_checkpoint1.pt')
metrics = {'val_accu': 98.5, 'train_accu': 99.2}
checkpoint = experiment.new_checkpoint(epoch=1, metrics=metrics)
checkpoint.id

'97f7fccf8d2d67aa78d7d58a86b5d938ddff9f38'

This returns us the checkpoint ID and tracks the path of the checkpoint created by the trainer.

Now let's say that we also want ModelBox to store the checkpoint, we will simply set the flag `upload` in the above api

In [12]:
#resp = checkpoint.upload_artifact(Artifact(path=checkpoint_path,mime_type=ArtifactMime.Checkpoint))

Once checkpoints are created they can be listed by passing the experiment name

In [12]:
experiment.list_checkpoints()

[Checkpoint(id='97f7fccf8d2d67aa78d7d58a86b5d938ddff9f38', experiment_id='e71a25a8d3d481463c281f083f5e2671ea2896bc', epoch=1, artifacts=[], metrics={})]

## Working with Models and ModelVersions

Model objects describe tasks performed, metadata, which datasets are used to train, how to use the models during inference, etc. ModelVersions are trained instances of a model. So for example over time an English ASR(speech to text) model can have multiple model versions as they are trained with different datasets and such. 

We don't prescribe the granularity of Models and ModelVersions. If it's easier to create different Models every time a new model is trained with different hyperparameters and a single ModelVersion pointing to the model artifacts and all the metrics that is fine.

In [5]:
model = mbox.new_model(name='asr_en', owner='owner@owner.org', namespace='langtech', task='asr', description='ASR for english', metadata={'x': 'y'})
model.id

'93b7cf0bec10ff6d500a9273bc34ab7eff02eeca'

In the same way a ModelVersion can be created by the client, and track the associated artifacts and metadata.


In [6]:
tags =["test"]
model_version = model.new_model_version(name="asr_en_july", version="1", description='ASR for english', metadata={'x': 'y'}, unique_tags=tags, artifacts=[], framework=MLFramework.PYTORCH)
model_version.id

'0ae662556996e18f364a41b82cd1a4e043062687'

Once a modelversion is created we can upload the model and associate with the model version object.

In [19]:
model_path = './artifacts/mnist_cnn.pt'
resp = model_version.upload_artifact(Artifact(parent=model_version.id, path=model_path, mime_type=ArtifactMime.Checkpoint))
resp

UploadArtifactResponse(id='5a0094ee34e88d826a84ef831444b152b9c2c91a')

The model file can now be served by the file server built into Model Box to inference servers. Inference services can either use the language specific SDKs in Python, Rust or Go or call the GRPC `DownloadFile` API directly which streams the files.

> **_NOTE: Checkpoints Transforms to ModelVersions_** 
Usually in production engineers look at checkpoints/models created during training and select a version which has the best metrics. Once we have the worker infrastructure in place, we will create APIs which to do automatic convertion of checkpoints to ModelVersions.

## Tracking Artifacts and Working with Files
Modelbox can track artifacts used in training and also users can upload Files and associate them with experiments, models and model versions. For example, a user can track the dataset files used for training stored in S3 or even upload them to ModelBox. A trained model can be uploaded and then later streamed to applications for inferencing.


In [7]:
import pathlib
file_path = str(
            pathlib.Path(".").parent.resolve().joinpath("artifacts/test_artifact.txt")
        )
artifacts = [Artifact(parent=model_version.id, mime_type=ArtifactMime.Text, path=file_path)]
resp = experiment.track_artifacts(artifacts=artifacts)
resp

TrackArtifactsResponse(num_artifacts_tracked=2)

Modelbox is now tracking the artifact and has information about the checksum, local path of the file, etc.

In [9]:
experiment.artifacts

[Artifact(parent='e71a25a8d3d481463c281f083f5e2671ea2896bc', path='/home/jovyan/work/tutorials/artifacts/test_artifact.txt', mime_type=<ArtifactMime.Text: 3>, checksum='0019d23bef56a136a1891211d7007f6f', id='7501e792880236c52f408070ab6934cb9f21b806')]

## Metrics 
ModelBox supports adding Metrics to experiments. Metrics can be logged to a key with values being a float, string or bytes. Metric values are associated with a step unit, and wallclock time when the metric was emitted by the application.

In [24]:
import time
experiment.log_metrics("val_accu", step=1, wallclock=int(time.time()), value=0.73)
experiment.log_metrics("val_accu", step=2, wallclock=int(time.time()), value=0.78)

experiment.get_all_metrics()

{'val_accu': [MetricValue(step=1, wallclock_time=1663836663, value=0.7300000190734863),
  MetricValue(step=2, wallclock_time=1663836663, value=0.7799999713897705)]}

Metrics can be added to any of the modelbox objects including Model, ModelVersion.